In [501]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import spacy
import numpy as np
from spacy.en import English
import string
import unicodedata

from collections import Counter
from itertools import chain

parser = English()



sqd_data = pd.read_json('/home/anjana/Downloads/train-v1.1.json') 

##### Prepare Stopwords list, question type mapping list, punctuation mark list #####

In [522]:
stopword = stopwords.words('english')
qtntype_list = ['What', 'When', 'Why', 'Which', 'Who','Whom', 'How']
pun_list = list(string.punctuation)
delelements_list = stopword + qtntype_list + pun_list
 

##### Considering one matter from SQuAD data set - (Third matter) ##### 

In [524]:
matter = sqd_data.data[0]['paragraphs'][3] 
matter

{u'context': u'The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study \u2013 aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering \u2013 with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A. and Master of Business Administration (MBA) degrees, respectively.',
 u'qas': [{u'answers': [{u'answer_start': 487, u'text': u'eight'}],
   u'id': u'5733a6424776f41900660f51',
   u'question': u'How many BS level degrees are offered in the College of Engineering at Notre Dame?'},
  {u'answers': [{u'answer_start': 46,

##### Extraction of  context #####

In [525]:
context = sqd_data.data[0]['paragraphs'][3]['context'] 
context = unicodedata.normalize('NFKD', context).encode('ascii','ignore')
context


'The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study  aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering  with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A. and Master of Business Administration (MBA) degrees, respectively.'

##### Context pre processing ##### 

###### Sentence tokenization of the context #####

In [526]:
senttokenized_context = sent_tokenize(context)
sentencelist = []
for i in senttokenized_context:
      sentencelist.append(i) 

In [527]:
senttokenized_context

['The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s.',
 'Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study  aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering  with eight B.S.',
 'degrees offered.',
 'Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A.',
 'and Master of Business Administration (MBA) degrees, respectively.']

##### Word tokenization, Lemmatisation, stop word removal #####

In [528]:
sentence_list = []
lemmatizer = WordNetLemmatizer()


for i in sentencelist:
    words = nltk.word_tokenize(i)
    words = ([x.lower() for x in words])
    words = ([x for x in words if x not in stopword])
    words = [lemmatizer.lemmatize(x) for x in words]
    words = [[s.encode('ascii') for s in words]]
    sentence_list.append(words)

    

##### Questions list ##### 

In [628]:
question_list = []
for i in range(len(sqd_data.data[0]['paragraphs'][3]['qas'])): 
    l1 = sqd_data.data[0]['paragraphs'][3]['qas'][i]['question'] 
    l1 = unicodedata.normalize('NFKD', l1).encode('ascii','ignore')

    question_list.append(l1) 

In [629]:
question_list[0] 

'How many BS level degrees are offered in the College of Engineering at Notre Dame?'

* Keeping question's first word in upper case 

In [630]:
import re

def replace_all(repls, str):
    # return re.sub('|'.join(repls.keys()), lambda k: repls[k.group(0)], str)                                     
    return re.sub('|'.join(re.escape(key) for key in repls.keys()),
                  lambda k: repls[k.group(0)], str)     

In [631]:
Question_list = []
for i in range(len(question_list)): 
      Question_list.append(replace_all({"how": "How", "when": "When","what": "What","whom": "Whom","which": "Which"}, question_list[i]) 
                          )

In [632]:
Question_list

['How many BS level degrees are offered in the College of Engineering at Notre Dame?',
 'In What year was the College of Engineering at Notre Dame formed?',
 'Before the creation of the College of Engineering similar studies were carried out at Which Notre Dame college?',
 'How many departments are within the Stinson-Remick Hall of Engineering?',
 'The College of Science began to offer civil engineering courses beginning at What time at Notre Dame?']

##### Mapping of questions #####

In [633]:
question_mapping= []
delquestionpart_list = []

for i in range(len(Question_list)):
    l2 = nltk.word_tokenize(Question_list[i])
    for i in range(len(l2)):
        if l2[i] in qtntype_list:
                combine_list = l2[i]+" "+l2[i+1]
                remove_list = combine_list.split() 
                l3 = [x for x in l2 if x not in remove_list]
                question_mapping.append(combine_list) 
                delquestionpart_list.append(l3) 

In [634]:
question_mapping

['How many', 'What year', 'Which Notre', 'How many', 'What time']

##### Converting question sentence into 'unicode' structure for NER #####

In [635]:
joined_delquestionpart_list =  []
for i in range(len(delquestionpart_list)):
        joined_question = unicode(" ".join(delquestionpart_list[i]),"utf-8") 
        joined_delquestionpart_list.append(joined_question) 

In [636]:
joined_delquestionpart_list


[u'BS level degrees are offered in the College of Engineering at Notre Dame ?',
 u'In was the College of Engineering at Notre Dame formed ?',
 u'Before the creation of the College of Engineering similar studies were carried out at Dame college ?',
 u'departments are within the Stinson-Remick Hall of Engineering ?',
 u'The College of Science began to offer civil engineering courses beginning at at Notre Dame ?']

In [637]:
import spacy
nlp = spacy.load('en')

qn_ner_mapping = []
for i in range(len(joined_delquestionpart_list)):
    doc = nlp(joined_delquestionpart_list[i])
    for ent in doc.ents:
        qn_ner_mapping.append([i,ent.text,ent.label_])  

In [638]:
qn_ner_mapping

[[0, u'the College of Engineering', u'ORG'],
 [0, u'Notre Dame', u'GPE'],
 [1, u'the College of Engineering', u'ORG'],
 [1, u'Notre Dame', u'GPE'],
 [2, u'the College of Engineering', u'ORG'],
 [2, u'Dame', u'GPE'],
 [3, u'Stinson', u'PERSON'],
 [4, u'The College of Science', u'ORG'],
 [4, u'Notre Dame', u'GPE']]

##### Data frame for QUESTION mapping #####

In [639]:
Question_no = []
Qn_word = [] 
Qn_tag = []
for i in range(len(qn_ner_mapping)):
    Question_no.append(qn_ner_mapping[i][0])
    Qn_word.append(qn_ner_mapping[i][1])
    Qn_tag.append(qn_ner_mapping[i][2])
     

In [640]:
question_mapping

['How many', 'What year', 'Which Notre', 'How many', 'What time']

In [641]:
for i in range(len(question_mapping)):
    df = question_mapping.index(question_mapping[i])
    print(df)

0
1
2
0
4


In [642]:
Question_character_df = pd.DataFrame()
qn_mapping = []
for i in range(len(Question_no)):
    for j in range(len(question_mapping)):
        if (Question_no[i]) == j:
            qn_mapping.append(question_mapping[j])


Question_character_df['QuestionNO'] = Question_no
Question_character_df['QuestionMapping'] = qn_mapping 
Question_character_df['Word'] = Qn_word
Question_character_df['WordTag'] = Qn_tag


##### Question - mapping #####

In [643]:
Question_character_df

,QuestionNO,QuestionMapping,Word,WordTag
0,0,How many,the College of Engineering,ORG
1,0,How many,Notre Dame,GPE
2,1,What year,the College of Engineering,ORG
3,1,What year,Notre Dame,GPE
4,2,Which Notre,the College of Engineering,ORG
5,2,Which Notre,Dame,GPE
6,3,How many,Stinson,PERSON
7,4,What time,The College of Science,ORG
8,4,What time,Notre Dame,GPE


################################################################################################################

##### Context preprocessing ##### 

In [644]:
context = sqd_data.data[0]['paragraphs'][3]['context'] 
context = unicodedata.normalize('NFKD', context).encode('ascii','ignore')
context

'The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s. Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study  aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering  with eight B.S. degrees offered. Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A. and Master of Business Administration (MBA) degrees, respectively.'

##### Sentence tokenization #####

In [645]:
senttokenized_context = sent_tokenize(context)
sentencelist = []
for i in senttokenized_context:
      sentencelist.append(i) 

In [646]:
senttokenized_context

['The College of Engineering was established in 1920, however, early courses in civil and mechanical engineering were a part of the College of Science since the 1870s.',
 'Today the college, housed in the Fitzpatrick, Cushing, and Stinson-Remick Halls of Engineering, includes five departments of study  aerospace and mechanical engineering, chemical and biomolecular engineering, civil engineering and geological sciences, computer science and engineering, and electrical engineering  with eight B.S.',
 'degrees offered.',
 'Additionally, the college offers five-year dual degree programs with the Colleges of Arts and Letters and of Business awarding additional B.A.',
 'and Master of Business Administration (MBA) degrees, respectively.']

In [647]:
contextword_list = []
lemmatizer = WordNetLemmatizer()

 
for i in sentencelist:
    words = nltk.word_tokenize(i)
    #words = ([x.lower() for x in words])
    words = ([x for x in words if x not in stopword])
    words = [lemmatizer.lemmatize(x) for x in words]
    words = [[s.encode('ascii') for s in words]]
    contextword_list.append(words)

    

In [558]:
contextword_list[0]

[['The',
  'College',
  'Engineering',
  'established',
  '1920',
  ',',
  'however',
  ',',
  'early',
  'course',
  'civil',
  'mechanical',
  'engineering',
  'part',
  'College',
  'Science',
  'since',
  '1870s',
  '.']]

##### Extracting answer words #####

In [648]:
ans_list = []
for i in range(len(sqd_data.data[0]['paragraphs'][3]['qas'])): 
    l1 = sqd_data.data[0]['paragraphs'][3]['qas'][i] 

    ans_word = l1['answers'][0]['text']
    ans_word = unicodedata.normalize('NFKD', ans_word).encode('ascii','ignore')
    ans_split = ans_word.split()
    #ans_words = [x.lower() for x in ans_split]
    ans_list.append(ans_split) 
    
    

In [649]:
ans_list

[['eight'],
 ['1920'],
 ['the', 'College', 'of', 'Science'],
 ['five'],
 ['the', '1870s']]

##### Converting answer words to 'unicode'  for NER #####

In [650]:
answers = []
for i in range(len(ans_list)):
    l1 = ' '.join(ans_list[i]) 
    l1 = unicode(l1,"utf-8")
    answers.append(l1) 

In [651]:
answers

[u'eight', u'1920', u'the College of Science', u'five', u'the 1870s']

In [652]:
ans_mapping = []
ans_word = []
for i in range(len(answers)):
    doc = nlp(answers[i])
    for ent in doc.ents:
        ans_mapping.append(ent.label_)  
        ans_word.append(ent.text)
        

##### Answer mapping ###### 

In [668]:
Answer_character_df = pd.DataFrame()
Answer_character_df['QuestionNo'] = range(len(question_mapping))
Answer_character_df['QuestionMapping'] = question_mapping
Answer_character_df['AnswerWord'] = ans_word
Answer_character_df['AnsWordTag'] = ans_mapping

In [669]:
Answer_character_df

,QuestionNo,QuestionMapping,AnswerWord,AnsWordTag
0,0,How many,eight,CARDINAL
1,1,What year,1920,DATE
2,2,Which Notre,the College of Science,ORG
3,3,How many,five,CARDINAL
4,4,What time,the 1870s,DATE


###################################################################################################################

##### Co occurance matrix #####

In [656]:
Answer_document = []
for i in range(len(Answer_character_df.index)): 
    Answer_document.append([Answer_character_df['QuestionMapping'][i],Answer_character_df['AnsWordTag'][i]]) 

In [657]:
Answer_document

[['How many', u'CARDINAL'],
 ['What year', u'DATE'],
 ['Which Notre', u'ORG'],
 ['How many', u'CARDINAL'],
 ['What time', u'DATE']]

In [658]:
Question_character_df.head()

,QuestionNO,QuestionMapping,Word,WordTag
0,0,How many,the College of Engineering,ORG
1,0,How many,Notre Dame,GPE
2,1,What year,the College of Engineering,ORG
3,1,What year,Notre Dame,GPE
4,2,Which Notre,the College of Engineering,ORG


In [659]:
Question_document = []
for i in range(len(Question_character_df.index)):
    Question_document.append([Question_character_df['QuestionMapping'][i],Question_character_df['WordTag'][i]])


In [660]:
Question_document

[['How many', u'ORG'],
 ['How many', u'GPE'],
 ['What year', u'ORG'],
 ['What year', u'GPE'],
 ['Which Notre', u'ORG'],
 ['Which Notre', u'GPE'],
 ['How many', u'PERSON'],
 ['What time', u'ORG'],
 ['What time', u'GPE']]

##### joining question-tag and answer-tag #####

In [661]:
document = Answer_document + Question_document

In [662]:
document


[['How many', u'CARDINAL'],
 ['What year', u'DATE'],
 ['Which Notre', u'ORG'],
 ['How many', u'CARDINAL'],
 ['What time', u'DATE'],
 ['How many', u'ORG'],
 ['How many', u'GPE'],
 ['What year', u'ORG'],
 ['What year', u'GPE'],
 ['Which Notre', u'ORG'],
 ['Which Notre', u'GPE'],
 ['How many', u'PERSON'],
 ['What time', u'ORG'],
 ['What time', u'GPE']]

In [664]:
import numpy
import itertools
from collections import Counter
varnames = tuple(sorted(set(itertools.chain(*document))))


In [665]:
expanded = [tuple(itertools.combinations(d, 2)) for d in document]
expanded = itertools.chain(*expanded)

# Sort the combinations so that A,B and B,A are treated the same
expanded = [tuple(sorted(d)) for d in expanded]

# count the combinations
c = Counter(expanded) 


# Create the table
table = numpy.zeros((len(varnames),len(varnames)), dtype=int)

for i, v1 in enumerate(varnames):
    for j, v2 in enumerate(varnames[i:]):        
        j = j + i 
        table[i, j] = c[v1, v2]
        table[j, i] = c[v1, v2]


In [666]:
co_occ_matrix = pd.DataFrame(table,index = varnames,columns = varnames)
co_occ_matrix

,CARDINAL,DATE,GPE,How many,ORG,PERSON,What time,What year,Which Notre
CARDINAL,0,0,0,2,0,0,0,0,0
DATE,0,0,0,0,0,0,1,1,0
GPE,0,0,0,1,0,0,1,1,1
How many,2,0,1,0,1,1,0,0,0
ORG,0,0,0,1,0,0,1,1,2
PERSON,0,0,0,1,0,0,0,0,0
What time,0,1,1,0,1,0,0,0,0
What year,0,1,1,0,1,0,0,0,0
Which Notre,0,0,1,0,2,0,0,0,0


In [671]:
np.log(1 + 0.8)

0.58778666490211906

In [673]:
np.log(1 + 0)

0.0

In [674]:
np.array(co_occ_matrix)

array([[0, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1],
       [2, 0, 1, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 2],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 2, 0, 0, 0, 0]])